In [1]:
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

2022-11-08 18:04:59.770042: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-08 18:04:59.968247: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-08 18:04:59.968267: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-08 18:05:00.009676: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-08 18:05:01.069981: W tensorflow/stream_executor/platform/de

Changement des paramètres d'affichages

In [2]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True,threshold=np.inf)
pd.options.display.max_rows = 999

#### Traitement des données

In [3]:
df = pd.read_csv("kddcup99_csv.csv")
col2encode = ["protocol_type","service","flag"]
ohe_col = pd.get_dummies(df[col2encode],columns=col2encode)
df = df.drop(columns=col2encode)
df_ohe = pd.concat([df,ohe_col],axis=1)
attack_values = pd.unique(df_ohe["label"].values.ravel())
attack_values = attack_values[1:] ## Remove "normal" as it is not an attack
### Create a dict to remap attack to True and normal to False
d_attack = {"normal":False}
for a in attack_values:
    d_attack.update({a:True})
label = df_ohe["label"].map(d_attack)
df_ohe = df_ohe.drop(columns="label",axis=1)
X_train, X_test, y_train, y_test = train_test_split(df_ohe, label, test_size=0.25, random_state=42)

#### Création du modèle

In [4]:
model = keras.Sequential(name="my_sequential")
model.add(keras.Input(shape=(118,)))
model.add(keras.layers.Dense(64,use_bias=True))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.ReLU())
model.add(keras.layers.Dense(64,use_bias=True))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.ReLU())
model.add(keras.layers.Dense(64,use_bias=True))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.ReLU())
model.add(keras.layers.Dense(1,use_bias=True))

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics =['accuracy']
)

2022-11-08 18:05:05.226881: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-08 18:05:05.226926: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-08 18:05:05.226966: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (emile-HVY-WXX9): /proc/driver/nvidia/version does not exist
2022-11-08 18:05:05.227588: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Entrainement

In [5]:
model.fit(X_train, y_train, batch_size=64, epochs=10,validation_data=(X_test,y_test))

Epoch 1/10


2022-11-08 18:05:05.618292: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 349766160 exceeds 10% of free system memory.


5790/5790 [==============================] - 22s 4ms/step - loss: 0.6749 - accuracy: 0.9095 - val_loss: 0.4341 - val_accuracy: 0.8267
Epoch 2/10
5790/5790 [==============================] - 21s 4ms/step - loss: 0.2888 - accuracy: 0.9578 - val_loss: 0.3734 - val_accuracy: 0.9738
Epoch 3/10
5790/5790 [==============================] - 22s 4ms/step - loss: 0.1986 - accuracy: 0.9714 - val_loss: 0.3054 - val_accuracy: 0.8413
Epoch 4/10
5790/5790 [==============================] - 21s 4ms/step - loss: 0.1742 - accuracy: 0.9769 - val_loss: 0.1308 - val_accuracy: 0.9552
Epoch 5/10
5790/5790 [==============================] - 21s 4ms/step - loss: 0.1359 - accuracy: 0.9799 - val_loss: 0.3995 - val_accuracy: 0.9685
Epoch 6/10
5790/5790 [==============================] - 21s 4ms/step - loss: 0.1137 - accuracy: 0.9794 - val_loss: 0.0728 - val_accuracy: 0.9853
Epoch 7/10
5790/5790 [==============================] - 21s 4ms/step - loss: 0.0984 - accuracy: 0.9821 - val_loss: 0.2544 - val_accuracy: 0.9

#### Test

In [6]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test)
print("test acc:", results)

Evaluate on test data
3860/3860 [==============================] - 6s 1ms/step - loss: 0.4605 - accuracy: 0.9204
test acc: [0.4605478048324585, 0.9203838109970093]
